Базовый метод СЛАУ

In [18]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla

def jacobi(A, b, x0, tol=1e-6, max_iterations=1000):
    n = len(b)
    D = np.diag(A)  # Extract diagonal of A
    R = A - np.diagflat(D)  # Remainder matrix
    
    x = x0.copy()
    for _ in range(max_iterations):
        x_new = (b - np.dot(R, x)) / D
        if np.linalg.norm(x_new - x) < tol:
            return x_new
        x = x_new
    return x

# Define the initial conditions for testing
A_initial = np.array([[4, -1, 0, 0],
                      [-1, 4, -1, 0],
                      [0, -1, 4, -1],
                      [0, 0, -1, 3]], dtype=float)

b_initial = np.array([15, 10, 10, 10], dtype=float)
x0_initial = np.zeros_like(b_initial)

solution_initial = jacobi(A_initial, b_initial, x0_initial)

# Create a diagonally dominant matrix A for better convergence
n = len(b_initial)
diagonals = [-1 * np.ones(n-1), 10 * np.ones(n), -1 * np.ones(n-1)]
A_dominant = sp.diags(diagonals, [-1, 0, 1], format="csr")

def jacobi_preconditioned(A, B_inv, b, x0, tol=1e-6, max_iterations=10000):
    n = len(b)
    D = A.diagonal()  # Extract diagonal of A
    R = A - sp.diags(D)  # Remainder matrix
    
    x = x0.copy()
    for k in range(max_iterations):
        # Solve the preconditioned system Bx_new = b - Rx
        x_new = B_inv.dot(b - R.dot(x))
        if np.linalg.norm(x_new - x) < tol:
            return x_new, k+1  # Return the solution and number of iterations
        x = x_new
    return x, k+1  # If it doesn't converge within max_iterations

def compute_ilu_B_inv(A):
    ilu_factorization = spla.spilu(A)
    # Define a function to perform the inverse operation of B using L and U
    def B_inv_action(v):
        y = spla.spsolve_triangular(ilu_factorization.L, v, lower=True)
        return spla.spsolve_triangular(ilu_factorization.U, y, lower=False)
    B_inv = spla.LinearOperator(A.shape, matvec=B_inv_action)
    return B_inv

# Compute B_inv for the new A using ILU
B_ilu_inv_dominant = compute_ilu_B_inv(A_dominant)

# Test the preconditioned Jacobi method with ILU preconditioner for the new A
solution_ilu_dominant, iterations_ilu_dominant = jacobi_preconditioned(A_dominant, B_ilu_inv_dominant, b_initial, x0_initial)

# Test the preconditioned Jacobi method with Identity matrix as preconditioner for the new A
#B_eye_inv = np.eye(n)
#solution_eye_dominant, iterations_eye_dominant = jacobi_preconditioned(A_dominant, B_eye_inv, b_initial, x0_initial)
# Generate a random matrix
B_random = np.random.rand(n, n)

# Ensure it's invertible by adding a large scalar to its diagonal (making it dominant)
B_random += np.eye(n) * 10

# Compute its inverse
B_random_inv = np.linalg.inv(B_random)

# Test the preconditioned Jacobi method with the random matrix as the preconditioner for the new A
solution_random_dominant, iterations_random_dominant = jacobi_preconditioned(A_dominant, B_random_inv, b_initial, x0_initial)



# Calculate epsilon errors
error_ilu = np.linalg.norm(A_dominant.dot(solution_ilu_dominant) - b_initial)
#error_eye = np.linalg.norm(A_dominant.dot(solution_eye_dominant) - b_initial)
error_random = np.linalg.norm(A_dominant.dot(solution_random_dominant) - b_initial)


#iterations_ilu_dominant, iterations_eye_dominant, error_ilu, error_eye, solution_ilu_dominant, solution_eye_dominant
iterations_ilu_dominant, iterations_random_dominant, error_ilu, error_random, solution_ilu_dominant, solution_random_dominant


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:485: SparseEfficiencyWarning: spilu converted its input to CSC format
  warn('spilu converted its input to CSC format',
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:640: SparseEfficiencyWarning: CSR matrix format is required. Converting to CSR matrix.
  warn('CSR matrix format is required. Converting to CSR matrix.',


(10,
 9,
 5.031965304193132,
 3.831329223564748,
 array([1.81161792, 1.49108403, 1.62509561, 1.47412748]),
 array([1.41924087, 1.10777768, 1.02135432, 0.85531031]))

In [22]:
# Increase the size of the matrix and vectors
n_large = 10_000

# Create a larger diagonally dominant matrix A for better convergence
diagonals_large = [-1 * np.ones(n_large-1), 10 * np.ones(n_large), -1 * np.ones(n_large-1)]
A_dominant_large = sp.diags(diagonals_large, [-1, 0, 1], format="csr")

# Larger b vector
b_large = np.random.rand(n_large)

# Initial guess
x0_large = np.zeros_like(b_large)

# Compute B_inv for the new large A using ILU
B_ilu_inv_large = compute_ilu_B_inv(A_dominant_large)

# Test the preconditioned Jacobi method with ILU preconditioner for the larger matrix
solution_ilu_large, iterations_ilu_large = jacobi_preconditioned(A_dominant_large, B_ilu_inv_large, b_large, x0_large)

# Generate a random matrix for the larger system
B_random_large = np.random.rand(n_large, n_large)
B_random_large += np.eye(n_large) * 10  # Ensure it's invertible
B_random_inv_large = np.linalg.inv(B_random_large)

# Test the preconditioned Jacobi method with the random matrix as the preconditioner for the larger matrix
solution_random_large, iterations_random_large = jacobi_preconditioned(A_dominant_large, B_random_inv_large, b_large, x0_large)

# Calculate epsilon errors for the larger system
error_ilu_large = np.linalg.norm(A_dominant_large.dot(solution_ilu_large) - b_large)
error_random_large = np.linalg.norm(A_dominant_large.dot(solution_random_large) - b_large)

iterations_ilu_large, iterations_random_large, error_ilu_large, error_random_large

/var/folders/r5/11z3453j76jc05_sqjgl82j40000gs/T/ipykernel_41278/353820171.py:43: RuntimeWarning: overflow encountered in subtract
  if np.linalg.norm(x_new - x) < tol:


(13, 10000, 17.32554136800965, nan)